### Congestion Prediction

Problem Statement:
We would predict congestion in the city of Bogor based on traffic data in that city

In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH_DATA='../data/raw/'
PATH_DATA_INTERIM='../data/interim/'

## Get The data

Preview data

In [3]:
data_agg_jams=pd.read_csv(PATH_DATA+'aggregate_median_jams_Kota Bogor.csv')

In [4]:
data_agg_jams.sample(3)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
43161,43161,2022-07-31 21:00:00.000,32.71,KOTA BOGOR,Haji Achmad Sobana,2,519.0,67.0,14.45,1,35097095,2022-07-31,2.0,"LINESTRING (106.807901 -6.580977, 106.808541 -..."
94710,15346,2022-08-31 20:00:00.000,32.71,KOTA BOGOR,N9 Raden KH Abdullah bin Nuh,1,1287.0,67.0,28.55,15,36957748,2022-08-31,1.0,"MULTILINESTRING ((106.77938 -6.556308, 106.779..."
102109,22745,2022-09-04 19:00:00.000,32.71,KOTA BOGOR,Brigjen Saptadji Hadi Prawira,1,1469.0,85.0,24.67,21,37225749,2022-09-04,1.0,"MULTILINESTRING ((106.763351 -6.564478, 106.76..."


In [5]:
data_agg_jams.shape

(102322, 14)

In [8]:
data_agg_jams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 102322 non-null  int64  
 1   time                       102322 non-null  object 
 2   kemendagri_kabupaten_kode  102322 non-null  float64
 3   kemendagri_kabupaten_nama  102322 non-null  object 
 4   street                     100529 non-null  object 
 5   level                      102322 non-null  int64  
 6   median_length              102322 non-null  float64
 7   median_delay               102322 non-null  float64
 8   median_speed_kmh           102322 non-null  float64
 9   total_records              102322 non-null  int64  
 10  id                         102322 non-null  int64  
 11  date                       102322 non-null  object 
 12  median_level               102322 non-null  float64
 13  geometry                   10

In [9]:
#Mengambil label target yaitu level dan feature yang dipakai
data_agg_jams2=data_agg_jams[['id','time','street','median_length','median_delay','median_speed_kmh','total_records','level']]

In [10]:
data_agg_jams2.sample(6)

,id,time,street,median_length,median_delay,median_speed_kmh,total_records,level
39861,35022436,2022-07-30 13:00:00.000,Pajajaran Indah 5,523.0,142.0,7.25,19,3
98304,37147355,2022-09-03 09:00:00.000,KH Soleh Iskandar (Jalur Lambat),631.0,122.0,11.72,1,3
55787,35603446,2022-08-08 07:00:00.000,Atang Sanjaya,938.0,63.0,17.62,2,1
61004,35773193,2022-08-11 13:00:00.000,Meranti,550.0,64.0,12.26,1,2
8486,33771976,2022-07-11 18:00:00.000,TB M Falak,404.0,88.0,10.47,11,2
20007,34263328,2022-07-18 11:00:00.000,N9 Jalan Raya Pajajaran,190.0,133.0,4.58,1,4


In [11]:
col_null = data_agg_jams2.isnull().sum()/len(data_agg_jams2)
col_null[col_null>0]

street    0.017523
dtype: float64

In [12]:
data_agg_jams2.dropna(inplace=True)

C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\2997304100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_agg_jams2.dropna(inplace=True)


In [13]:
data_agg_jams2.shape

(100529, 8)

In [14]:
#menambahkan kolom streettime untuk foreign key untuk menggabungkan feature baru yang akan dibuat
data_agg_jams2['time2'] = pd.to_datetime(data_agg_jams2['time'])
data_agg_jams2['hours'] = data_agg_jams2['time2'].dt.strftime('%H').astype(int)
data_agg_jams2['is_weekend'] = data_agg_jams2['time2'].dt.strftime('%w').isin(['0','6']).astype(int)
data_agg_jams2['streettime']= data_agg_jams2['street']+'_'+data_agg_jams2['hours'].astype(str)+'_'+data_agg_jams2['is_weekend'].astype(str)

C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\3523374544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_agg_jams2['time2'] = pd.to_datetime(data_agg_jams2['time'])
C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\3523374544.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_agg_jams2['hours'] = data_agg_jams2['time2'].dt.strftime('%H').astype(int)
C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\3523374544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [16]:
train_size = int(len(data_agg_jams2)*0.8) ##--> train size
print(train_size)

80423


In [17]:
congestion_train = data_agg_jams2.sample(train_size,random_state = 103)['id']
data_agg_jams2['is_train'] = data_agg_jams2.id.isin(congestion_train).astype(int)

C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\390412219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_agg_jams2['is_train'] = data_agg_jams2.id.isin(congestion_train).astype(int)


In [18]:
data_agg_jams2.sample()

,id,time,street,median_length,median_delay,median_speed_kmh,total_records,level,time2,hours,is_weekend,streettime,streettime2,is_train
85497,36639691,2022-08-26 23:00:00.000,N8 Jalan Raya Sukabumi,689.0,83.0,16.24,9,2,2022-08-26 23:00:00,23,0,N8 Jalan Raya Sukabumi_23_0,N8 Jalan Raya Sukabumi_2022-08-26 23:00:00.000,1


## Waktu Features
menggunakan kolom streettime, time2, level, hours, is_weekend pada data data_agg_jams2
1. rata-rata level pada weekdays (senin-jumat)
2. simpangan baku level pada weekdays (senin-jumat)
3. rata-rata level pada weekend
4. simpangan baku pada weekend
5. rata-rata level pada jam berangkat kerja (jam 6-9) pada hari kerja
6. simpangan baku pada jam berangkat kerja (jam 6-9) pada hari kerja
7. rata-rata level pada jam pulang kerja (jam 15-18) pada hari kerja
8. simpangan baku pada jam pulang kerja (jam 15-18) pada hari kerja

In [19]:
a=data_agg_jams2[data_agg_jams2.is_train==1]
street_date=a[['streettime','time2','level','hours','is_weekend']]

In [20]:
street_date.sample(10)

,streettime,time2,level,hours,is_weekend
31554,N9 Raden KH Abdullah bin Nuh_11_0,2022-07-25 11:00:00,1,11,0
87401,Pangeran Asogiri_18_1,2022-08-27 18:00:00,2,18,1
523,N9 Jalan Raya Pajajaran_12_0,2022-07-06 12:00:00,4,12,0
55309,Brigjen Saptadji Hadi Prawira_18_1,2022-08-07 18:00:00,2,18,1
24756,Cilebut Raya_14_0,2022-07-21 14:00:00,2,14,0
83415,Cilebut Raya_17_0,2022-08-25 17:00:00,2,17,0
17983,Kapten Muslihat_12_1,2022-07-17 12:00:00,3,12,1
5190,Jalan Kampung Buntar_12_1,2022-07-09 12:00:00,3,12,1
17097,Kedunghalang Raya_19_1,2022-07-16 19:00:00,1,19,1
44194,Brigjen Saptadji Hadi Prawira_14_0,2022-08-01 14:00:00,3,14,0


In [21]:
street_date['level_weekend'] = street_date.apply(lambda x: x['level']
                        if x['is_weekend'] == 1 else np.nan, axis=1)
street_date['level_weekday'] = street_date.apply(lambda x: x['level']
                        if x['is_weekend'] == 0 else np.nan, axis=1)

C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\1080592666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  street_date['level_weekend'] = street_date.apply(lambda x: x['level']
C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\1080592666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  street_date['level_weekday'] = street_date.apply(lambda x: x['level']


In [22]:
street_date['weekday69']= street_date.apply(lambda x: x['level']
                        if x['is_weekend']==0 and (x['hours']==6 or x['hours']==7 or x['hours']==8 or x['hours']==9)
                        else np.nan, axis=1)
street_date['weekday1518']= street_date.apply(lambda x: x['level']
                        if x['is_weekend']==0 and (x['hours']==15 or x['hours']==16 or x['hours']==17 or x['hours']==18)
                        else np.nan, axis=1)

C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\2494142457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  street_date['weekday69']= street_date.apply(lambda x: x['level']
C:\Users\ramad\AppData\Local\Temp\ipykernel_10588\2494142457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  street_date['weekday1518']= street_date.apply(lambda x: x['level']


In [23]:
street_date.sample(5)

,streettime,time2,level,hours,is_weekend,level_weekend,level_weekday,weekday69,weekday1518
13140,N6 Jalan Raya Kemang-Bogor_20_0,2022-07-14 20:00:00,2,20,0,NaN,2.0,NaN,NaN
102296,Semplak Raya_22_1,2022-09-04 22:00:00,2,22,1,2.0,NaN,NaN,NaN
68231,Laladon Raya_14_0,2022-08-15 14:00:00,3,14,0,NaN,3.0,NaN,NaN
19460,KH Soleh Iskandar (Jalur Lambat)_4_0,2022-07-18 04:00:00,5,4,0,NaN,5.0,NaN,NaN
62159,N9 Jalan Raya Tajur_9_0,2022-08-12 09:00:00,2,9,0,NaN,2.0,2.0,NaN


In [24]:
streetWeekend = street_date[street_date['is_weekend']==1]
streetWeekend = streetWeekend[['streettime','level_weekend']]

In [25]:
#Weekend Mean
weekendmean=streetWeekend.groupby('streettime').level_weekend.mean().reset_index()
weekendmean.columns=['streettime','weekendmean']
weekendmean.sample(4)

,streettime,weekendmean
2553,Warung Nangka_12_1,1.000000
1168,Kebon Pedes_20_1,2.230769
1954,Pangeran Asogiri_10_1,1.736842
1823,Pahlawan_18_1,2.720930


In [26]:
#Weekend Std
weekendstd=streetWeekend.groupby('streettime').level_weekend.std().reset_index()
weekendstd.columns=['streettime','weekendstd']
weekendstd.sample(4)

,streettime,weekendstd
2405,Tawakal_16_1,NaN
473,Ciomas Raya_19_1,0.922266
2075,RE Abdullah_10_1,0.739458
1198,Kenanga Permai_20_1,NaN


In [27]:
streetWeekdays = street_date[street_date['is_weekend']==0]
streetWeekdays = streetWeekdays[['streettime','level_weekday']]

In [28]:
#Weekdays Mean
weekdaysmean=streetWeekdays.groupby('streettime').level_weekday.mean().reset_index()
weekdaysmean.columns=['streettime','weekdaysmean']
weekdaysmean.sample(4)

,streettime,weekdaysmean
2196,RE Abdullah_6_0,3.021277
2258,Raden Kosasih_15_0,2.000000
1212,Kencana 2 Raya_14_0,2.200000
1807,N9 Raden KH Abdullah bin Nuh_17_0,2.595588


In [29]:
#Weekdays Mean
weekdaysstd=streetWeekdays.groupby('streettime').level_weekday.std().reset_index()
weekdaysstd.columns=['streettime','weekdaysstd']
weekdaysstd.sample(4)

,streettime,weekdaysstd
1280,Lawang Gintung_12_0,1.414214
1133,Kapten Yusuf_7_0,1.095726
1948,Pakuan_19_0,0.534522
562,Dewi Sartika_19_0,0.816497


In [30]:
streetWeekdays69= street_date[street_date.weekday69.notnull()]
streetWeekdays69 = streetWeekdays69[['streettime','weekday69']]

In [31]:
#Weekdays jam 6-9 Mean
weekdaysmean69=streetWeekdays69.groupby('streettime').weekday69.mean().reset_index()
weekdaysmean69.columns=['streettime','weekdaysmean69']
weekdaysmean69.sample(4)

,streettime,weekdaysmean69
235,Mandala Raya_8_0,3.200000
371,Pengadilan_9_0,3.250000
107,Gang Aut_7_0,3.500000
290,N9 Jalan Raya Dramaga_7_0,2.711538


In [32]:
#Weekdays jam 6-9 Std
weekdaysstd69=streetWeekdays69.groupby('streettime').weekday69.std().reset_index()
weekdaysstd69.columns=['streettime','weekdaysstd69']
weekdaysstd69.sample(4)

,streettime,weekdaysstd69
234,Mandala Raya_7_0,1.178511
106,Exit Tol Tajur_7_0,NaN
109,Gang Puma_6_0,NaN
423,Sempur Kaler_8_0,NaN


In [33]:
streetWeekdays1518 = street_date[street_date.weekday1518.notnull()]
streetWeekdays1518 = streetWeekdays1518[['streettime','weekday1518']]

In [34]:
#Weekdays jam 15-18 mean
weekdaysmean1518=streetWeekdays1518.groupby('streettime').weekday1518.mean().reset_index()
weekdaysmean1518.columns=['streettime','weekdaysmean1518']
weekdaysmean1518.sample(4)

,streettime,weekdaysmean1518
536,RE Abdullah_18_0,2.695652
616,Tajur Indah_15_0,4.000000
338,Lebak Nangka_15_0,3.000000
323,Kolonel Enjo Martadisastra_18_0,3.000000


In [35]:
#Weekdays jam 16-19 std
weekdaysstd1518=streetWeekdays1518.groupby('streettime').weekday1518.std().reset_index()
weekdaysstd1518.columns=['streettime','weekdaysstd1518']
weekdaysstd1518.sample(4)

,streettime,weekdaysstd1518
240,Jalan Empang_17_0,0.753678
371,Mayor Oking_18_0,0.000000
244,Jalan Kampung Buntar_17_0,0.500000
102,Ciherang Raya_16_0,NaN


In [36]:
time_feature = [weekendmean, weekendstd, weekdaysmean, weekdaysstd, weekdaysmean69, weekdaysstd69, weekdaysmean1518, weekdaysstd1518]

## Merge Features

In [37]:
df_interim = data_agg_jams2.copy()

for d in time_feature:
    df_interim = pd.merge(df_interim, d, on = 'streettime', how = 'left')


In [38]:
df_interim.to_csv(PATH_DATA_INTERIM+'iter1_interim.csv',index = False)